# Fetch songs of EG
This notebook was used to retrieve the texts for the EG songs.

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import pyarrow as pa
import pyarrow.parquet as pq

BASE_URL = "https://www.evangeliums.net/lieder"
SONG_LIST_URL = f"{BASE_URL}/liederbuch_evangelisches_gesangbuch.html"


In [92]:

def get_song_links():
    response = requests.get(SONG_LIST_URL)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    
    songs = []
    # Find all links in the song list
    for link in soup.select('table a'):
        title = link.text.strip()
        href = link.get('href')
        if "/lieder" in href:
            continue
        if href and title:
            full_url = BASE_URL + '/' + href
            songs.append({"title": title, "url": full_url})
    return songs
    
def get_song_text(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    main = soup.find('main')
    print(f"found main")
    if not main:
        return None

    # Step 1: First <div> inside <main>
    divs = main.find_all('div', 'card-body')
    print(f"found {len(divs)} divs")
    if len(divs) < 1:
        return None
    song_text_div = divs[-1]

    print(f"found classes: {song_text_div.get('class', [])}")
    if not song_text_div or 'card-body' not in song_text_div.get('class', []):
        return None

    # Clean up: Remove all inner <div> elements inside the song_text_div
    for inner_div in song_text_div.find_all('div'):
        inner_div.decompose()

    # Extract all <p> tags
    verses = [
        p.get_text().replace("<br>", " | ").replace("\r\n", " | ")
        for p in song_text_div.find_all('p')
        if p.get_text(strip=True)
    ]

    # Join verses with "|"
    song_text = " | ".join(verses)
    return song_text


In [93]:
songs = get_song_links()
print(f"Found {len(songs)} songs.")


Found 536 songs.


### Try out function `get_song_text`

In [94]:
song = songs[0]
get_song_text(song['url'])

found main
found 3 divs
found classes: ['card-body']


"1)  Macht hoch die Tür, die Tor macht weit; | es kommt der Herr der Herrlichkeit, | ein König aller Königreich, | ein Heiland aller Welt zugleich, | der Heil und Leben mit sich bringt; | derhalben jauchzt, mit Freuden singt: | Gelobet sei mein Gott, | mein Schöpfer reich von Rat. | 2)  Er ist gerecht, ein Helfer wert; | Sanftmütigkeit ist sein Gefährt, | sein Königskron ist Heiligkeit, | sein Zepter ist Barmherzigkeit; | all unsre Not zum End er bringt, | derhalben jauchzt, mit Freuden singt: | Gelobet sei mein Gott, | mein Heiland groß von Tat. | 3)  O wohl dem Land, o wohl der Stadt, | so diesen König bei sich hat. | Wohl allen Herzen insgemein, | da dieser König ziehet ein. | Er ist die rechte Freudensonn, | bringt mit sich lauter Freud und Wonn. | Gelobet sei mein Gott, | mein Tröster früh und spat. | 4)  Macht hoch die Tür, die Tor macht weit, | eu'r Herz zum Tempel zubereit'. | Die Zweiglein der Gottseligkeit | steckt auf mit Andacht, Lust und Freud; | so kommt der König auch zu

### Retrieve all song texts

In [ ]:
songs = get_song_links()
print(f"Found {len(songs)} songs.")

data = []
for idx, song in enumerate(songs):
    print(f"Fetching ({idx + 1}/{len(songs)}): {song['title']}")
    try:
        text = get_song_text(song['url'])
        if text:
            data.append({
                "title": song['title'],
                "book": "EG",
                "number": idx+536,
                "lyrics": text
            })
        else:
            print(f"Warning: No text found for {song['title']}")
    except Exception as e:
            print(f"Error fetching {song['title']}: {e}")
    
    # Be polite to the server
    time.sleep(0.5)

df = pd.DataFrame(data)
print(f"Saving {len(df)} songs to 'evangelisches_gesangbuch_wu.parquet'...")

df.to_parquet("evangelisches_gesangbuch.parquet", engine="pyarrow", index=False)

Found 148 songs.
Fetching (1/148): Auf auf ihr Christen alle
found main
found 3 divs
found classes: ['card-body']
Fetching (2/148): Sieh, dein König kommt zu dir
found main
found 3 divs
found classes: ['card-body']
Fetching (3/148): Der Tag, der ist so freudenreich
found main
found 3 divs
found classes: ['card-body']
Fetching (4/148): Sieh nicht an, was du selber bist
found main
found 3 divs
found classes: ['card-body']
Fetching (5/148): Stern über Bethlehem
found main
found 3 divs
found classes: ['card-body']
Fetching (6/148): Von guten Mächten wunderbar geborgen
found main
found 3 divs
found classes: ['card-body']
Fetching (7/148): Bleib bei uns, wenn der Tag entweicht
found main
found 3 divs
found classes: ['card-body']
Fetching (8/148): Geh unter der Gnade (Alte Stunden, alte Tage)
found main
found 3 divs
found classes: ['card-body']
Fetching (9/148): Wie schön leuchtet der Morgenstern
found main
found 3 divs
found classes: ['card-body']
Fetching (10/148): Ach mein Herr Jesu
found 

In [ ]:
# restart the loop with this SONG_LIST_URL to retrieve the songs for Regionalteil Württemberg
SONG_LIST_URL = f"{BASE_URL}/liederbuch_evangelisches_gesangbuch_regionalteil_wuerttemberg.html"

In [ ]:
pd.read_parquet("evangelisches_gesangbuch.parquet")

,title,book,number,lyrics
0,Macht hoch die Tür,EG,1,"1) Macht hoch die Tür, die Tor macht weit; | ..."
1,Er ist die rechte Freudensonn (Kanon),EG,2,"1) Er ist die rechte Freudensonn, | bringt mi..."
2,"Gott, heilger Schöpfer aller Stern",EG,3,"1) Gott, heil'ger Schöpfer aller Stern, | erl..."
3,"Nun komm, der Heiden Heiland",EG,4,"1) Nun komm der Heiden Heiland, | der Jungfrau..."
4,Gottes Sohn ist kommen,EG,5,1) Gottes Sohn ist kommen | uns allen zu From...
...,...,...,...,...
528,"Auferstehn, ja auferstehn wirst du",EG,678,"1) Auferstehn, ja auferstehn wirst du, | mein ..."
529,"Du kamst, du gingst mit leiser Spur",EG,679,"Du kamst, du gingst mit leiser Spur, | ein flü..."
530,"Brich herein, süßer Schein",EG,680,"1) Brich herein, süßer Schein selger Ewigkeit!..."
531,Wir sind nur Gast auf Erden,EG,681,1) Wir sind nur Gast auf Erden | und wandern o...


In [ ]:
# merge existing songs database (FJ songs) with EG songs
df0 = pd.read_parquet("songs.parquet")
df1 = pd.read_parquet("evangelisches_gesangbuch.parquet")
df2 = pd.concat([df0, df1], axis=0)
df2.to_parquet("songs_with_eg.parquet")